Key steps for assignment:
- Get the data for the comparison
- Get the models used for hte comparison
- Train the models with suggested hyperparameters (compare the test results with a given number of epochs)
- OR train the models until convergence and compare the resutls (number of steps and test results)
- Hyperparameter tuninig for a fixed number of trials
- Compare the improvements with tuning

Tasks:
- image classification (CIFAR10, CIFAR100) -> ResNet18, ResNet34, ResNet50 (but freaking huge)
- image recognition (ImageNet (maybe)) -> ResNet18 or ResNet50 (ImageNet more pain in the ass to download)
- Denosing (SIDD dataset) -> UNet or ResUnet
- GAN (CIFAR10) -> WassersteinGAN
- Language modelling (Penn TreeBank)

In [2]:
! pip install madgrad
! pip install adabelief-pytorch==0.2.0
! pip install optuna

You should consider upgrading via the 'c:\users\lucaz\anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\lucaz\anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\lucaz\anaconda3\python.exe -m pip install --upgrade pip' command.


Import relevant libraries

In [36]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from adabelief_pytorch import AdaBelief
from madgrad import MADGRAD

In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### DATA

In [31]:
data = "CIFAR10"
# data = "CIFAR100"
# data = "ImageNet"

if data == "CIFAR10":
    transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    trainset= torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                              shuffle=True)
    testset= torchvision.datasets.CIFAR10(root='./data', train=False,
                                           download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                             shuffle=False)
if data == "CIFAR100":
    trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                              shuffle=True)
    testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                           download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                             shuffle=False)

# For now not working. Need to download the data locally first
if data == "ImageNet":
    trainset = torchvision.datasets.ImageNet(root='./data', train=True,
                                        download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                              shuffle=True)
    testset = torchvision.datasets.ImageNet(root='./data', train=False,
                                           download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                             shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [34]:
# Choose the model to use (uncomment the line)

model = models.resnet18()
# model = models.resnet34()

In [37]:
# Loss and optimizers with suggested hyperparameters for CIFAR
criterion = nn.CrossEntropyLoss()
# optimizer = AdaBelief(model.parameters(), lr=0.001, betas=(0.9,0.999), eps=1e-16, weight_decouple=False, rectify=False) #fixed_decay=False, amsgrad=False, weight_decay=5e-4
optimizer = MADGRAD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0, eps=1e-6, decouple_decay=False)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

Train the model

In [38]:
# Train the model for image classification
model.to(device)
num_epochs = 200
total_step = len(trainloader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(trainloader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/391] Loss: 1.6398
Epoch [1/5], Step [200/391] Loss: 1.6111
Epoch [1/5], Step [300/391] Loss: 1.3022
Epoch [2/5], Step [100/391] Loss: 1.3663
Epoch [2/5], Step [200/391] Loss: 1.1516
Epoch [2/5], Step [300/391] Loss: 1.1648
Epoch [3/5], Step [100/391] Loss: 0.8393
Epoch [3/5], Step [200/391] Loss: 1.1248
Epoch [3/5], Step [300/391] Loss: 0.8392
Epoch [4/5], Step [100/391] Loss: 1.0720
Epoch [4/5], Step [200/391] Loss: 0.7634
Epoch [4/5], Step [300/391] Loss: 0.6591
Epoch [5/5], Step [100/391] Loss: 0.7404
Epoch [5/5], Step [200/391] Loss: 0.5967
Epoch [5/5], Step [300/391] Loss: 0.6569


In [39]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in testloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))

Accuracy of the model on the test images: 70.88 %
